In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [3]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

2113536/2110848 [==============================] - 0s 0us/step


In [4]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)
y_train = y_data[:1395]
y_test = y_data[1395:]

## 1. Deep RNN
- RNNs can be made deep, with multiple layers, like CNNs or MLPs
- Beware that RNNs take long to train compared to CNNs

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [6]:
def deep_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [7]:
model = KerasClassifier(build_fn = deep_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 2s 55ms/step - loss: 3.2571 - accuracy: 0.6803
Epoch 2/200
28/28 [==============================] - 2s 55ms/step - loss: 2.0078 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 2s 56ms/step - loss: 1.3712 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 2s 55ms/step - loss: 1.2368 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 2s 56ms/step - loss: 1.1963 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 2s 56ms/step - loss: 1.1817 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 2s 56ms/step - loss: 1.1727 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 2s 56ms/step - loss: 1.1684 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 2s 56ms/step - loss: 1.1651 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 2s 56ms/step - loss: 1.1636 - accuracy: 0.7147

In [8]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
12/12 [==============================] - 0s 12ms/step


In [9]:
print(accuracy_score(y_pred, y_test_))

0.8497495826377296


## 2. Bidirectional RNN
- Bidirectional RNNs consider not only one-way influence of sequence, but also the other way
- Actually, they can be thought as building two separate RNNs, and merging them\
<br>
<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/bidirectional-rnn.png" style="width: 400px"/>
</br>

In [10]:
from tensorflow.keras.layers import Bidirectional

In [11]:
def bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (49,1)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [12]:
model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 1s 19ms/step - loss: 3.5629 - accuracy: 0.2330
Epoch 2/200
28/28 [==============================] - 1s 19ms/step - loss: 2.4295 - accuracy: 0.7039
Epoch 3/200
28/28 [==============================] - 1s 18ms/step - loss: 1.3877 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 1s 19ms/step - loss: 1.1779 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 1s 20ms/step - loss: 1.1300 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 1s 20ms/step - loss: 1.0874 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 1s 20ms/step - loss: 1.0424 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 1s 18ms/step - loss: 0.9979 - accuracy: 0.7161
Epoch 9/200
28/28 [==============================] - 0s 18ms/step - loss: 0.9545 - accuracy: 0.7226
Epoch 10/200
28/28 [==============================] - 1s 18ms/step - loss: 0.9078 - accuracy: 0.7548

In [13]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

12/12 [==============================] - 0s 5ms/step


In [14]:
print(accuracy_score(y_pred, y_test_))

0.8430717863105175


## 3. Deep Bidirectional RNN
- Bidirectional RNNs can be stacked

<img src="http://www.wildml.com/wp-content/uploads/2015/09/Screen-Shot-2015-09-16-at-2.21.51-PM-272x300.png" style="width: 300px"/>

In [15]:
def deep_bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(10, return_sequences = True), input_shape = (49,1)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = False)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [16]:
model = KerasClassifier(build_fn = deep_bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 2s 72ms/step - loss: 3.5133 - accuracy: 0.2330
Epoch 2/200
28/28 [==============================] - 2s 72ms/step - loss: 2.4088 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 2s 71ms/step - loss: 1.5577 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 2s 71ms/step - loss: 1.2687 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 2s 74ms/step - loss: 1.2092 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 2s 73ms/step - loss: 1.1894 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 2s 72ms/step - loss: 1.1794 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 2s 73ms/step - loss: 1.1736 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 2s 72ms/step - loss: 1.1697 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 2s 73ms/step - loss: 1.1661 - accuracy: 0.7147

In [17]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

12/12 [==============================] - 0s 14ms/step


In [18]:
print(accuracy_score(y_pred, y_test_))

0.8447412353923205
